In [ ]:
import os

In [ ]:
! ls

drive  Experiments  periodic_behavior_bn_wd  sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! git clone https://github.com/tipt0p/periodic_behavior_bn_wd.git

fatal: destination path 'periodic_behavior_bn_wd' already exists and is not an empty directory.


In [ ]:
! python periodic_behavior_bn_wd/run_train_and_test.py

bash: activate: No such file or directory


In [ ]:
ls periodic_behavior_bn_wd

checkpoints/   fake.py            nets/               README.md
CIFAR10/       get_info_funcs.py  parser_get_info.py  run_train_and_test.py*
cycle_env.yml  get_info.py        parser_train.py     training_utils.py
data.py        LICENSE            Plots.ipynb         train.py
Experiments/   nb_utils.py        __pycache__/


In [ ]:
! ls

drive  Experiments  periodic_behavior_bn_wd  sample_data


In [ ]:
! ls periodic_behavior_bn_wd

checkpoints    fake.py		  nets		      README.md
CIFAR10        get_info_funcs.py  parser_get_info.py  run_train_and_test.py
cycle_env.yml  get_info.py	  parser_train.py     training_utils.py
data.py        LICENSE		  Plots.ipynb	      train.py
Experiments    nb_utils.py	  __pycache__


In [ ]:
os.chdir("./periodic_behavior_bn_wd")

In [ ]:
ls

checkpoints/   fake.py            nets/               README.md
CIFAR10/       get_info_funcs.py  parser_get_info.py  run_train_and_test.py*
cycle_env.yml  get_info.py        parser_train.py     training_utils.py
data.py        LICENSE            Plots.ipynb         train.py
Experiments/   nb_utils.py        __pycache__/


In [ ]:
! python ./run_train_and_test.py

bash: activate: No such file or directory


In [ ]:
import os
import subprocess
import argparse
import datetime
import time

parser = argparse.ArgumentParser(description='Managing experiments')
parser.add_argument('--test', action='store_true',
                        help='print (test) or os.system (run)')
parser.add_argument('-f', help='service')

args = parser.parse_args()
    
if args.test:
    action = print
else:
    action = os.system

ENVIRONMENT = 'CycleBNNet_env'
 
data = 'CIFAR10'#'CIFAR10''CIFAR100'
network = 'ConvNetSI'#'ResNet18SI''ConvNetSI''ResNet18SIAf''ConvNetSIAf'
fix_noninvlr = 0.0

save_path = './Experiments/{}_{}/'.format(network,data)
if fix_noninvlr >=0:
    save_path = './Experiments/{}_{}_noninvlr_{}/'.format(network,data,fix_noninvlr) 
if not os.path.exists('./Experiments'):
    os.mkdir('./Experiments')
if not os.path.exists(save_path):
    os.mkdir(save_path)

params = {'dataset' : data,
          'model': network,
          'noninvlr':fix_noninvlr,
          'momentum': 0.0,
          'num_channels':32,
          'depth':3,# work only for ConvNet
          'epochs': 1001,
          'corrupt_train': 0.0,
          'save_freq': 1,
          'eval_freq':1000,
          'use_data_size':50000,
          'dir': save_path + 'checkpoints',
          'init_scale':10.,
          'fix_si_pnorm_value':-1,
          'gpu':0
         }

lrs = [0.01,]    
wds = [0.001,]

add_params = '--use_test --no_schedule --no_aug'#--fbgd --fix_si_pnorm

params_test = {'dataset' : data,
          'model': network,
          'num_channels': params['num_channels'],
          'depth': params['depth'],
          'init_scale':params['init_scale'],
          'save_path': save_path + 'info',
          'models_dir': save_path + 'checkpoints',
          'use_data_size':params['use_data_size'],
          'gpu':params['gpu']
         }

log_path = save_path + 'logs/'
if not os.path.exists(save_path):
    os.mkdir(save_path)
if not os.path.exists(log_path):
    os.mkdir(log_path)
    
info_path = save_path + 'info/'
if not os.path.exists(info_path):
    os.mkdir(info_path)

commands = []

for ind in range(len(lrs)):
    p = params.copy()
    p['lr_init'] = lrs[ind]
    p['wd'] = wds[ind]

    p_test = params_test.copy()

    exp_name = 'c{}_d{}_ds{}_lr{}_wd{}_mom{}_corr{}_epoch{}'.format(p['num_channels'],p['depth'],p['use_data_size'],p['lr_init'],p['wd'],p['momentum'],p['corrupt_train'],p['epochs'])
    if 'no_schedule' in add_params:
        exp_name = exp_name + '_nosch'
    if p['init_scale'] >0:
        exp_name = exp_name + 'initscale{}'.format(p['init_scale'])
    if 'no_aug' in add_params:
        exp_name = exp_name + '_noaug'
    if 'fbgd' in add_params:
        exp_name = exp_name + '_fbgd'
    if 'fix_si_pnorm' in add_params:
        exp_name = exp_name + '_fix_si_pnorm{}'.format(p['fix_si_pnorm_value'])
        

    p['dir'] = params['dir'] + '/' + exp_name
    exp_log_path = log_path + exp_name

    p_test['models_dir'] = params_test['models_dir'] + '/' + exp_name + '/trial_0'

    # train
    command = 'train.py {} >> {}'.format(' '.join(["--{} {}".format(k,v) for (k, v) in p.items()])+' ' +add_params, exp_log_path+'.out')
    commands.append(command)

    #train metrics
    p_test['save_path'] = params_test['save_path'] + '/' + exp_name + '/train-tm.npz'
    commands.append('get_info.py {} --corrupt_train {} --train_mode --eval_model --all_pnorm'.format(' '.join(["--{} {}".format(k,v) for (k, v) in p_test.items()]), p['corrupt_train']))
    commands.append('get_info.py {} --corrupt_train {} --train_mode --update  --calc_grad_norms'.format(' '.join(["--{} {}".format(k,v) for (k, v) in p_test.items()]), p['corrupt_train']))

    #test metrics
    p_test['save_path'] = params_test['save_path'] + '/' + exp_name + '/test-em.npz'
    commands.append('get_info.py {} --use_test --eval_model'.format(' '.join(["--{} {}".format(k,v) for (k, v) in p_test.items()])))
           
                    
if ENVIRONMENT:
    tmp_str = ' && ~/anaconda3/envs/{}/bin/python '.format(ENVIRONMENT)
    final_command = "bash -c '. activate {} {} {}'".format(ENVIRONMENT,tmp_str,tmp_str.join(commands))
else:
    final_command = 'python '.join(command)


                  

In [ ]:
import argparse

def parser():
    parser = argparse.ArgumentParser(description="Model training")
    
    parser.add_argument(
        '--gpu', 
        type=str,
        default='0',
        help="GPU to use"
    )
    
    parser.add_argument(
        "--dir",
        type=str,
        default="./checkpoints",
        required=False,
        help="training directory (default: None)",
    )
    
    parser.add_argument(
        "--dataset", 
        type=str, 
        default="CIFAR10", 
        help="dataset name (default: CIFAR10)"
    )
    
    parser.add_argument(
        "--data_path",
        type=str,
        default="./CIFAR10",
        metavar="PATH",
        help="path to datasets location (default: ~/datasets/)",
    )
    
    parser.add_argument(
        "--use_test",
        dest="use_test",
        action="store_true",
        help="use test dataset instead of validation (default: False)",
    )
    
    parser.add_argument(
        "--corrupt_train", 
        type=float, 
        default=None,
        help="train data corruption fraction (default: None --- no corruption)",
    )
    
    parser.add_argument(
        "--split_classes", 
        type=int, 
        default=None,
        help="split classes for CIFAR-10 (default: None --- no split)",
    )

    parser.add_argument(
        "--fbgd",
        dest="fbgd",
        action="store_true",
        help="train with full-batch GD (default: False)",
    )
    
    parser.add_argument(
        "--batch_size",
        type=int,
        default=128,
        metavar="N",
        help="input batch size (default: 128)",
    )
    
    parser.add_argument(
        "--num_workers",
        type=int,
        default=4,
        metavar="N",
        help="number of workers (default: 4)",
    )
    
    parser.add_argument(
        "--model",
        type=str,
        default="ConvNetSI",
        required=False,
        metavar="MODEL",
        help="model name (default: None)",
    )
    
    parser.add_argument(
        "--trial",
        type=int,
        default=0,
        help="trial number (default: 0)",
    )
    
    parser.add_argument(
        "--resume_epoch",
        type=int,
        default=-1,
        metavar="CKPT",
        help="checkpoint epoch to resume training from (default: -1 --- start from scratch)",
    )

    parser.add_argument(
        "--epochs",
        type=int,
        default=1001,
        metavar="N",
        help="number of epochs to train (default: 1001)",
    )
    
    parser.add_argument(
        "--use_data_size",
        type=int,
        default=None,
        help="how many examples to use for training",
    )
    
    parser.add_argument(
        "--save_freq",
        type=int,
        default=None, 
        metavar="N",
        help="save frequency (default: None --- custom saving)",
    )
    
    parser.add_argument(
        "--save_freq_int",
        type=int,
        default=0, 
        metavar="N",
        help="internal save frequency - how many times to save at each epoch (default: None --- custom saving)",
    )
    
    parser.add_argument(
        "--eval_freq",
        type=int,
        default=10,
        metavar="N",
        help="evaluation frequency (default: 10)",
    )
    
    parser.add_argument(
        "--lr_init",
        type=float,
        default=0.1,
        metavar="LR",
        help="initial learning rate (default: 0.01)",
    )
    
    parser.add_argument(
        "--noninvlr",
        type=float,
        default=-1,
        metavar="LR",
        help="learning rate for not scale-inv parameters",
    )
    
    parser.add_argument(
        "--momentum",
        type=float,
        default=0.9,
        metavar="M",
        help="SGD momentum (default: 0.9)",
    )
    
    parser.add_argument(
        "--wd", 
        type=float, 
        default=1e-4, 
        help="weight decay (default: 1e-4)"
    )
    
    parser.add_argument(
        "--loss",
        type=str,
        default="CE",
        help="loss to use for training model (default: Cross-entropy)",
    )

    parser.add_argument(
        "--seed", 
        type=int, 
        default=1, 
        metavar="S", 
        help="random seed (default: 1)"
    )
    
    parser.add_argument(
        "--num_channels", 
        type=int, 
        default=64, 
        help="number of channels for resnet"
    )
    
    parser.add_argument(
        "--depth", 
        type=int, 
        default=3, 
        help="depth of convnet"
    )

    parser.add_argument(
        "--scale", 
        type=int, 
        default=25, 
        help="scale of lenet"
    )
    
    parser.add_argument(
        "--no_schedule", 
        default=True,
        action="store_true", 
        help="disable lr schedule"
    )
    
    parser.add_argument(
        "--c_schedule", 
        type=float,
        default=None, 
        help="continuous schedule - decrease lr linearly after 1/4 epochs so that at the end it is x times lower "
    )
    
    parser.add_argument(
        "--d_schedule", 
        type=float,
        default=None, 
        help="discrete schedule - decrease lr x times after each 1/4 epochs "
    )
    
    parser.add_argument(
        "--init_scale", 
        type=float,
        default=-1, 
        help="init norm of the last layer weights "
    )
    
    parser.add_argument(
        "--no_aug", 
        action="store_true", 
        help="disable data augmentation"
    )
    
    parser.add_argument(
        "--fix_si_pnorm", 
        action="store_true", 
        help="set SI-pnorm to init after each iteration"
    )
    
    parser.add_argument(
        "--fix_si_pnorm_value", 
        type=float, 
        default=-1,
        help="custom fixed SI-pnorm value (must go with --fix_si_pnorm flag; default: -1 --- use init SI-pnorm value)",
    )
    
    parser.add_argument(
        "--cosan_schedule", 
        action="store_true", 
        help="cosine anealing schedule"
    )

    parser.add_argument(
        "-f",  
        help="google colab arg"
    )

    args = parser.parse_args()
    return args

In [ ]:
import math
import torch
import torch.nn.functional as F
import numpy as np
import os, sys
import time
import tabulate
import data
import training_utils
import nets as models

columns = ["ep", "lr", "tr_loss", "tr_acc", "weights_norm", "te_loss", "te_acc", "time"]

def cross_entropy(model, input, target, reduction="mean"):
    "standard cross-entropy loss function"
    output = model(input)
    loss = F.cross_entropy(output, target, reduction=reduction)
    return loss, output

def check_si_name(n, model_name='ResNet18'):
    if model_name == 'ResNet18':
        return "conv1" in n or "1.bn1" in n or "1.0.bn1" in n or (("conv2" in n or "short" in n) and "4" not in n)
    elif model_name == 'ResNet18SI':
        return 'linear' not in n
    elif model_name == 'ResNet18SIAf':
        return ('linear' not in n and 'bn' not in n and 'shortcut.0' not in n)
    elif 'ConvNet' in model_name:
        return 'conv_layers.0.' in n or 'conv_layers.3.' in n or 'conv_layers.7.' in n or 'conv_layers.11.' in n
    return False
    
def main():
    
    args = parser()
    args.device = None
    
    os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
    os.environ['CUDA_VISIBLE_DEVICES']=args.gpu

    if torch.cuda.is_available():
        args.device = torch.device("cuda")
        args.cuda = True
    else:
        args.device = torch.device("cpu")
        args.cuda = False
  
    torch.backends.cudnn.benchmark = True
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)

    # n_trials = 1
    
    print("Preparing base directory %s" % args.dir)
    os.makedirs(args.dir, exist_ok=True)

    # for trial in range(n_trials):
    trial = args.trial
    output_dir = args.dir + f"/trial_{trial}"
    
    ### resuming is modified!!!
    if args.resume_epoch > -1:
        resume_dir = output_dir
        output_dir = output_dir + f"/from_{args.resume_epoch}_for_{args.epochs}"
        if args.save_freq_int > 0:
            output_dir = output_dir + f"_save_int_{args.save_freq_int}"
        if args.noninvlr >= 0:
            output_dir = output_dir + f"_noninvlr_{args.noninvlr}"
    ### resuming is modified!!!
    print("Preparing directory %s" % output_dir)

    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, "command.sh"), "w") as f:
        f.write(" ".join(sys.argv))
        f.write("\n")

    print("Using model %s" % args.model)
    model_cfg = getattr(models, args.model)

    print("Loading dataset %s from %s" % (args.dataset, args.data_path))
    transform_train = model_cfg.transform_test if args.no_aug else model_cfg.transform_train
    loaders, num_classes = data.loaders(
        args.dataset,
        args.data_path,
        args.batch_size,
        args.num_workers,
        transform_train,
        model_cfg.transform_test,
        use_validation=not args.use_test,
        use_data_size = args.use_data_size,
        split_classes=args.split_classes,
        corrupt_train=args.corrupt_train
    )

    print("Preparing model")
    print(*model_cfg.args)

    # add extra args for varying names
    if 'ResNet18' in args.model:
        extra_args = {'init_channels':args.num_channels}
        if "SI" in args.model:
            extra_args.update({'linear_norm':args.init_scale})
    elif 'ConvNet' in args.model:
        extra_args = {'init_channels':args.num_channels, 'max_depth':args.depth,'init_scale':args.init_scale}
    elif args.model == 'LeNet':
        extra_args = {'scale':args.scale}
    else:
        extra_args = {}
        

    model = model_cfg.base(*model_cfg.args, num_classes=num_classes, **model_cfg.kwargs, **extra_args)
    model.to(args.device)

    
    param_groups = model.parameters()

    if args.noninvlr >= 0:
        param_groups = [
            {'params': [p for n, p in model.named_parameters() if check_si_name(n, args.model)]},  # SI params are convolutions
            {'params': [p for n, p in model.named_parameters() if not check_si_name(n, args.model)],'lr':args.noninvlr},  # other params
        ]

    optimizer = torch.optim.SGD(param_groups, 
                                lr=args.lr_init, 
                                momentum=args.momentum, 
                                weight_decay=args.wd)
    
    if args.cosan_schedule:
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.epochs)

    epoch_from = args.resume_epoch + 1
    epoch_to = epoch_from + args.epochs
    print(f"Training from {epoch_from} to {epoch_to - 1} epochs")

    if epoch_from > 0:
        # Warning: due to specific lr schedule, resuming is generally not recommended!
        print(f"Loading checkpoint from the {args.resume_epoch} epoch")
        state = training_utils.load_checkpoint(resume_dir, args.resume_epoch)
        model.load_state_dict(state['state_dict'])
        optimizer.load_state_dict(state['optimizer'])
        if args.noninvlr >= 0:
            optimizer.param_groups[1]["lr"] = args.noninvlr
        
    si_pnorm_0 = None
    if args.fix_si_pnorm:
        if args.fix_si_pnorm_value > 0:
            # No lr schedule, plz...
            si_pnorm_0 = args.fix_si_pnorm_value
        else:
            si_pnorm_0 = np.sqrt(sum((p ** 2).sum().item() for n, p in model.named_parameters() if check_si_name(n, args.model)))
            
        print(f"Fixing SI-pnorm to value {si_pnorm_0:.4f}")


    for epoch in range(epoch_from, epoch_to):
        train_epoch(model, loaders, cross_entropy, optimizer, 
                    epoch=epoch, 
                    end_epoch=epoch_to, 
                    eval_freq=args.eval_freq, 
                    save_freq=args.save_freq,
                    save_freq_int=args.save_freq_int,
                    output_dir=output_dir,
                    lr_init=args.lr_init,
                    lr_schedule=not args.no_schedule,
                    noninvlr=args.noninvlr,
                    c_schedule=args.c_schedule,
                    d_schedule=args.d_schedule,
                    si_pnorm_0=si_pnorm_0,
                    fbgd=args.fbgd,
                    cosan_schedule = args.cosan_schedule)  
        if args.cosan_schedule:
            scheduler.step()
    

    print("model ", trial, " done")


def train_epoch(model, loaders, criterion, optimizer, epoch, end_epoch,
                eval_freq=1, save_freq=10, save_freq_int=0, output_dir='./', lr_init=0.01,
                lr_schedule=True, noninvlr = -1, c_schedule=None, d_schedule=None, si_pnorm_0=None,fbgd=False, 
               cosan_schedule = False):

    time_ep = time.time()

    if not cosan_schedule:
        if not lr_schedule:
            lr = lr_init
        elif c_schedule > 0:
            lr = training_utils.c_schedule(epoch, lr_init, end_epoch, c_schedule)
        elif d_schedule > 0:
            lr = training_utils.d_schedule(epoch, lr_init, end_epoch, d_schedule)
        else:
            lr = training_utils.schedule(epoch, lr_init, end_epoch, swa=False)
        if noninvlr >= 0:
            training_utils.adjust_learning_rate_only_conv(optimizer, lr)
        else:
            training_utils.adjust_learning_rate(optimizer, lr)
    else:
        for param_group in optimizer.param_groups:
            lr = param_group["lr"]
            break

    #train_res = training_utils.train_epoch(loaders["train"], model, criterion, optimizer, fbgd=fbgd,si_pnorm_0=si_pnorm_0,
    #                                       save_freq_int=save_freq_int,epoch = epoch,output_dir = output_dir)
    train_res = training_utils.SAM_train_epoch(loaders["train"], model, criterion, optimizer, r=0.05, fbgd=fbgd,si_pnorm_0=si_pnorm_0,
                                           save_freq_int=save_freq_int,epoch = epoch,output_dir = output_dir)
    if (
        epoch == 0
        or epoch % eval_freq == eval_freq - 1
        or epoch == end_epoch - 1
    ):
        test_res = training_utils.eval(loaders["test"], model, criterion)
    else:
        test_res = {"loss": None, "accuracy": None}
        
    def save_epoch(epoch):
        training_utils.save_checkpoint(
            output_dir,
            epoch,
            state_dict=model.state_dict(),
            optimizer=optimizer.state_dict(),
            train_res=train_res,
            test_res=test_res
        )

    if save_freq is None:
        if training_utils.do_report(epoch):
            save_epoch(epoch)
    elif epoch % save_freq == 0:
        save_epoch(epoch)
        
    time_ep = time.time() - time_ep
    values = [
        epoch,
        lr,
        train_res["loss"],
        train_res["accuracy"],
        train_res["weights_norm"],
        test_res["loss"],
        test_res["accuracy"],
        time_ep,
    ]
    table = tabulate.tabulate([values], columns, tablefmt="simple", floatfmt="8.4f")
    if epoch % 40 == 0:
        table = table.split("\n")
        table = "\n".join([table[1]] + table)
    else:
        table = table.split("\n")[2]
    print(table)

if __name__ == '__main__':
    main()


Preparing base directory ./checkpoints
Preparing directory ./checkpoints/trial_0
Using model ConvNetSI
Loading dataset CIFAR10 from ./CIFAR10
Files already downloaded and verified
Using train (45000) + validation (5000)
Files already downloaded and verified
Preparing model

Training from 0 to 1000 epochs
----  --------  ---------  --------  --------------  ---------  --------  --------
  ep        lr    tr_loss    tr_acc    weights_norm    te_loss    te_acc      time
----  --------  ---------  --------  --------------  ---------  --------  --------
   0    0.1000    14.5218   16.8244        229.7722     2.5711   26.0000   26.2192
   1    0.1000     2.5029   25.8800        222.4153                        25.1591
   2    0.1000     2.0535   31.8733        215.3984                        24.7349
   3    0.1000     1.8637   36.8289        208.7348                        24.7852
   4    0.1000     1.7570   39.8867        202.3719                        24.9820
   5    0.1000     1.8058   40

In [ ]:
5

In [ ]:
import os
import subprocess
import argparse
import datetime
import time

parser = argparse.ArgumentParser(description='Managing experiments')
parser.add_argument('--test', action='store_true',
                        help='print (test) or os.system (run)')
parser.add_argument('-f', help='service')

args = parser.parse_args()
    
if args.test:
    action = print
else:
    action = os.system

ENVIRONMENT = 'CycleBNNet_env'